In [73]:
import tensorflow as tf #библиотека с открытым исходным кодом для машинного обучения, предоставляет собой инструменты 
#для создания и тренировки нейронных сетей и других алгоритмов машинного обучения
from tensorflow import keras #высокоуровневая API для создания и обучения нейронных сетей, интегрированая в TensorFlow
#Используется для построения и тренировки моделей.
from tensorflow.keras import layers #позволяет использовать различные слои, доступные в Keras
import numpy as np #библиотека для работы с многомерными массивами и матрицами, а также для выполнения математических
#операций над ними,помогает обрабатывать и манипулировать данными
import json # для загрузки вопросов из датасета
import os #библиотека для взаимодействия с операционной системой


def create_image_model(input_shape): # создание функции create_image_model, которая принимает один параметр — input_shape
    #он указывает, какого размера будут входные данные
    inputs = layers.Input(shape=input_shape)# создание входного слоя, который будет принимать изображения в указанном формате
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs) # обавление свёрточного слоя, который использует 32 фильтра 
    #размером 3x3,он помогает модели находить важные детали в изображениях
    x = layers.MaxPooling2D(pool_size=(2, 2))(x) # добавление слоя подвыборки, он уменьшает размер изображения в два раза,
    #сохраняя только самое важное
    x = layers.Conv2D(64, (3, 3), activation='relu')(x) # свёрточный слой, с 64 фильтрами, этот слой ищет детали в изображении
    x = layers.MaxPooling2D(pool_size=(2, 2))(x) # слой подвыборки для дальнейшего уменьшения размера данных
    x = layers.Flatten()(x) # преобразование многомерных данных (которые получились после всех свёрток и подвыборок) в 
    #одномерный вектор, для того чтобы передать данные в следующий слой
    x = layers.Dense(128, activation='relu')(x) # добавление полносвязного слоя с 128 нейронами для того, чтобы модели 
    #могли делать более сложные выводы на основе извлеченных признаков
    model = keras.Model(inputs, x) # создание модели, которая связывает входные данные с выходом для понимания, как 
    #будет выглядеть модель
    return model # возвращение созданной модели, чтобы ее можно было использовать для обучения и предсказаний

def create_text_model(vocab_size, embedding_dim, input_length): # функция create_text_model принимает три параметра:
#vocab_size — количество уникальных слов в словаре
#embedding_dim — размерность векторного представления каждого слова
#input_length — длина входных текстов (количество слов в одном тексте)
    
    inputs = layers.Input(shape=(input_length,)) # создание входного слоя, который будет принимать последовательности 
    #слов фиксированной длины input_length
    x = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs) # слой встраивания (Embedding), он 
    #преобразует каждое слово в вектор фиксированной длины (embedding_dim)
    # использует словарь размером vocab_size, чтобы сопоставить каждое слово с его векторным представлением
    x = layers.LSTM(128)(x) # добавление слоя LSTM (долгая краткосрочная память) с 128 нейронами, он помогает модели 
    #обрабатывать последовательные данные, такие как текст, и запоминать контекст
    model = keras.Model(inputs, x) # создание модели, которая связывает входные данные с выходом (вектором, полученным из LSTM)
    return model # возвращение созданной модели, чтобы ее можно было использовать для обучения и предсказаний

def create_combined_model(image_model, text_model):  # функция create_combined_model принимает два параметра:
# image_model — модель для обработки изображений,
# text_model — модель для обработки текстов
    
    combined_input = layers.concatenate([image_model.output, text_model.output]) # объединение выхода обеих моделей 
    #(изображений и текста) в один общий вектор, что позволяет модели использовать информацию из обеих источников одновременно
    x = layers.Dense(64, activation='relu')(combined_input) # добавление полносвязного слоя с 64 нейронами и функцией 
    #активации ReLU, он помогает модели извлекать более сложные паттерны из объединенных данных
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 
    model = keras.Model(inputs=[image_model.input, text_model.input], outputs=outputs) # Ссздание выходноого слоя с 
    #одним нейроном и функцией 
    # активации сигмоиды(математическая функция, которая используется в нейронных сетях для преобразования выходных 
    #значений нейронов в диапазон от 0 до 1), 
    # он будет использоваться для бинарной классификации, то есть для предсказания двух классов
    return model# возвращение созданной модели, чтобы ее можно было использовать для обучения и предсказаний

image_input_shape = (64, 64, 3)  # входных изображений, которые будут подаваться в модель, каждое изображение будет 
#иметь размер 64 на 64 пикселя, и будет в цвете (RGB), что обозначается третьим значением 3
vocab_size = 1000  # размер словаря, модель будет учитывать только 1000 уникальных слов
embedding_dim = 64  # размерность векторного представления (эмбеддинга) для слов в тексте, каждое слово будет 
#представлено в виде вектора размером 64
max_question_length = 20  # максимальная длина вопросов, которые будут обрабатываться моделью, модель будет 
#принимать только вопросы длиной до 20 слов


image_model = create_image_model(image_input_shape) # создание модели для обработки изображений, используя 
#заданную форму входных изображений image_input_shape, которая равна (64, 64, 3)
# image_model будет представлять собой нейронную сеть, способную обрабатывать и анализировать изображения
text_model = create_text_model(vocab_size, embedding_dim, max_question_length) # создание модели для обработки 
#текста, используя параметры словаря, размерности эмбеддинга и максимальной длины вопросов
# text_model будет представлять собой нейронную сеть, способную обрабатывать текстовые данные и извлекать из них смысл
combined_model = create_combined_model(image_model, text_model) # создает комбинированную модель, которая объединяет 
#как модель для обработки изображений, так и модель для обработки текста
# combined_model будет представлять собой полную модель, способную одновременно обрабатывать как визуальные, так и 
#текстовые данные


combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # compile подготавливает 
#модель к обучению
# определяет, как модель будет обновлять свои веса во время обучения и какие метрики будут использоваться для оценки 
#её производительности
# optimizer='adam' оптимизатор Adam адаптивно изменяет скорость обучения для каждого параметра
# loss='binary_crossentropy' функция потерь, которая будет использоваться для оценки качества предсказаний модели
# metrics=['accuracy'] метрики, которые будут отслеживаться во время обучения и оценки модели, accuracy измеряет долю
#правильных предсказаний модели по сравнению с общим числом предсказаний


def load_clevr_questions(data_dir): # функция load_clevr_questions принимает аргумент data_dir, этот аргумент 
    #представляет собой путь к директории
    questions = [] # пустой список questions, который будет использоваться для хранения загруженных вопросов из набора данных
    answers = [] # пустой список answers, который будет использоваться для хранения ответов на вопросы

    for split in ['CLEVR_train', 'CLEVR_val']: # цикл for, чтобы пройти по двум значениям: 'CLEVR_train' и 'CLEVR_val'
        questions_file = os.path.join(data_dir, f'{split}_questions.json') # создается полное имя файла с вопросами, 
        #используя os.path.join, чтобы правильно объединить путь к директории data_dir
        if not os.path.exists(questions_file): # проверяется существование файла с вопросами по указанному пути, если 
            #файл не найден, то условие not os.path.exists(questions_file) будет истинным
            raise FileNotFoundError(f'Файл не найден: {questions_file}') # если файл не найден, будет вызвано исключение 
            #FileNotFoundError, и будет выведено сообщение о том, какой именно файл не найден

        with open(questions_file, 'r') as f: #  with автоматически закроет файл после завершения блока кода, даже если 
            #возникнет ошибка
            # open(questions_file, 'r') открывает файл, путь к которому хранится в переменной questions_file,
            #в режиме чтения ('r')
            # as f присваивает открытый файл переменной f, чтобы можно было ссылаться на него внутри блока with
            question_data = json.load(f) # функция json.load() загружает содержимое файла f в формате JSON и 
            #преобразовать его в соответствующий объект Python
             #question_data теперь будет содержать данные, которые были в файле questions_file

        for item in question_data['questions']: # for проходится по каждому элементу в списке questions, который 
            #находится в загруженных данных question_data
            # question_data —словарь, содержащий ключ 'questions', значение которого является списком, содержащим 
            #словари с вопросами и ответами
            questions.append(item['question']) # внутри цикла идет обращение к текущему элементу item, который 
            #представляет собой словарь, содержащий информацию о вопросе и ответе
            # item['question'] извлекает текст вопроса из текущего элемента, и добавляет его в список questions с 
            #помощью метода append(), это собирает все вопросы в одном списке
            answers.append(item['answer']) # идет извлечение ответа из текущего элемента с помощью item['answer'] и 
            #добавление его в список answers
            # после выполнения этого цикла будет два списка: один с вопросами и другой с соответствующими ответами

    return questions, answers # возвращение двух списков: один с вопросами и другой с ответами для того, чтобы 
    #использовать эти списки в других частях программы


CLEVR_DATA_DIR = 'C:/Users/Dasha/Desktop/CLEVR_v1.0' # путь к папке с данными

try: # в блоке try программа выполняет функцию load_clevr_questions, которая загружает вопросы
    #и ответы из указанной папки
    questions, answers = load_clevr_questions(CLEVR_DATA_DIR) # в переменные questions и answers 
    #записываются загруженные данные
    
    print(f"Количество вопросов: {len(questions)}")
    print(f"Пример вопроса: {questions[0]}")
    print(f"Пример ответа: {answers[0]}")
    
except FileNotFoundError as e: # 
    print(e) # 
except json.JSONDecodeError: # 
    print("Ошибка при загрузке данных: файл не является корректным JSON.") # 
except Exception as e: # 
    print(f"Произошла ошибка: {e}") # 


from sklearn.preprocessing import LabelEncoder # импортируется LabelEncoder из библиотеки sklearn, которая 
#используется для обработки данных

label_encoder = LabelEncoder() # создание объекта label_encoder, который будет
#использоваться для преобразования
#текстовых ответов в числовые значения
encoded_answers = label_encoder.fit_transform(answers) # метод fit_transform для преобразования
#списка answers в числовой формат

num_samples = 1000 # количество примеров, которые генерируются для тренировки модели
x_image_train = np.random.rand(num_samples, *image_input_shape)  # генерируются случайные числа для
#изображений, которые будут использоваться для тренировки модели
# используется функция np.random.rand из библиотеки NumPy, которая генерирует случайные числа
#в диапазоне от 0 до 1
# num_samples - количество примеров, которое генерируется
# *image_input_shape - это форма входного изображения
x_text_train = np.random.randint(0, vocab_size, size=(num_samples, max_question_length))
# генерируются случайные целые числа для текстовых данных, которые будут использоваться для тренировки модели
# используется функция np.random.randint из библиотеки NumPy, которая генерирует случайные целые числа
#в диапазоне от 0 до vocab_size
# num_samples - количество примеров, которое генерируется
# max_question_length - максимальная длина текстового вопроса
# vocab_size - количество уникальных слов в словаре (vocabularly)
y_train = np.random.randint(0, 2, size=(num_samples,))  # генерируются случайные метки (labels)
#для тренировки модели
# используется функция np.random.randint из библиотеки NumPy, которая генерирует случайные целые
#числа в диапазоне от 0 до 1


combined_model.fit([x_image_train, x_text_train], y_train, epochs=10, batch_size=32) 
# x_image_train и x_text_train - это входные данные для модели, они представляют
#собой изображения и текстовые данные
# y_train - это метки (labels) для данных, которые были сгенерированы, они представляют
#собой классификацию данных (0 или 1)
# epochs=10 - это количество эпох для обучения модели, одна эпоха - это один проход по всем данным
# batch_size=32 - это количество примеров, которые обрабатываются за один шаг


combined_model.summary() # информация о структуре модели combined_model для понимания,
как работает модель и какие слои она содержит


Количество вопросов: 849980
Пример вопроса: Are there more big green things than large purple shiny cubes?
Пример ответа: yes
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.4663 - loss: 0.8079
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.5343 - loss: 0.6867
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7528 - loss: 0.5664
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8796 - loss: 0.2771
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9697 - loss: 0.1222
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9857 - loss: 0.0370
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 1.0000 - loss: 0.0094
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 1.0000 - loss: 0.0028
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 1.0000 - loss: 5.7490e-04
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 1.0000 - loss: 2.9540e-04


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 62, 62,    │        896 │ input_layer_14[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 31, 31,    │          0 │ conv2d_14[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 29, 29,    │     18,496 │ max_pooling2d_14… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_15    │ (None, 14, 14,    │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 12544)     │          0 │ max_pooling2d_15… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 20, 64)    │     64,000 │ input_layer_15[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 128)       │  1,605,760 │ flatten_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 128)       │     98,816 │ embedding_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_7       │ (None, 256)       │          0 │ dense_21[0][0],   │
│ (Concatenate)       │                   │            │ lstm_7[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 64)        │     16,448 │ concatenate_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 1)         │         65 │ dense_22[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,413,445 (20.65 MB)

 Trainable params: 1,804,481 (6.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,608,964 (13.77 MB)